In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn import preprocessing #to transform the feature labels
from sklearn.feature_extraction import DictVectorizer #to turn categorial variables into numeric arrays
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer

In [2]:
train_set = pd.read_csv("Lab5_train.csv")
test_set = pd.read_csv("Lab5_test.csv")
train_set.head()

PassengerID  Survived  Pclass                     Name     Sex   Age  \
0         1302         0       3     Youseff, Mr. Gerious    male  45.5   
1          369         0       2  Chapman, Mr. John Henry    male  37.0   
2         1128         0       3     Petroff, Mr. Nedelio    male  19.0   
3          491         0       2        Mack, Mrs. (Mary)  female  57.0   
4         1087         0       3      Olsson, Miss. Elina  female  31.0   

   SibSp  ParCh       Ticket     Fare Cabin Embarked  
0      0      0         2628   7.2250   NaN        C  
1      1      0  SC/AH 29037  26.0000   NaN        S  
2      0      0       349212   7.8958   NaN        S  
3      0      0  S.O./P.P. 3  10.5000   E77        S  
4      0      0       350407   7.8542   NaN        S

Checking majority in train set

In [3]:
len(train_set[train_set["Survived"] == 1])/len(train_set) * 100

38.68852459016394

In [4]:
len(train_set[train_set["Survived"] == 0])/len(train_set) * 100

61.31147540983607

So, approx 61% didnt survive and 39% survived.

Extract Features: Using Pclass, Sex, Age, SubSp, parCh to train SVM 

In [5]:
train_set_features = train_set[['Pclass', 'Sex', 'Age', 'SibSp', 'ParCh','Survived']]
test_set_features = test_set[['Pclass', 'Sex', 'Age', 'SibSp', 'ParCh']]

In [6]:
train_set_features['Age'].fillna(train_set_features['Age'].mean(), inplace=True)
test_set_features['Age'].fillna(test_set_features['Age'].mean(), inplace=True)

/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [7]:
target = train_set_features.Survived

Encoding categorical features

In [8]:
vec = DictVectorizer()
train_set_non_class = vec.fit_transform(train_set_features[['Pclass', 'Sex', 'SibSp', 'ParCh']].to_dict('records')).toarray()
test_set_non_class = vec.fit_transform(test_set_features[['Pclass', 'Sex', 'SibSp', 'ParCh']].to_dict('records')).toarray()

Scale feature values

In [9]:
min_max_scaler = preprocessing.MinMaxScaler()
train_set_age_scaled = min_max_scaler.fit_transform(train_set_features.Age)
test_set_age_scaled = min_max_scaler.fit_transform(test_set_features.Age)

/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/rohitraghavan/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X

In [13]:
train_set_age_scaled_non_class = np.concatenate((train_set_non_class, np.reshape(train_set_age_scaled,(915,1))), axis=1)
test_set_age_scaled_non_class = np.concatenate((test_set_non_class, np.reshape(test_set_age_scaled,(394,1))), axis=1)

Train machine learning classifier

In [14]:
clf = svm.SVC(kernel='rbf')
clf.fit(train_set_age_scaled_non_class, target)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [23]:
target_predicted = clf.predict(test_set_age_scaled_non_class)

Write to csv

In [40]:
output = pd.DataFrame({"PassengerId": test_set.PassengerID.values, 
              "Survived": target_predicted})
output.to_csv("submission.csv", index=False)